### Installation

In [1]:
# if (!requireNamespace("BiocManager", quietly = TRUE))
#      install.packages("BiocManager")
#   BiocManager::install("cicero")

Vignette: https://www.bioconductor.org/packages/devel/bioc/vignettes/cicero/inst/doc/website.html

### Import packages

In [2]:
library(cicero)
library(data.table)
library(Matrix)
library(proxy)
library(reshape2)
library(umap)

Loading required package: monocle
Loading required package: Matrix
Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:Matrix’:

    colMeans, colSums, rowMeans, rowSums, which

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank,

In [3]:
datafr = readRDS('./datafr.rds')

In [4]:
dim(datafr)

[1] 385237  12178

In [5]:
mat_sparse = as(datafr, "dgTMatrix")
cicero_data = data.frame(cbind(Peak=rownames(datafr)[mat_sparse@i+1],
                               Cell=colnames(datafr)[mat_sparse@j+1],
                               Count=mat_sparse@x),stringsAsFactors = FALSE)
cicero_data$Count = as.numeric(cicero_data$Count)

In [6]:
head(cicero_data)

Peak,Cell,Count
chr1_3254750_3255907,AGCGATAGAATACGATAATGGCAGCTCGCAGGACGT,2
chr1_3378202_3378888,AGCGATAGAATACGATAATGGCAGCTCGCAGGACGT,2
chr1_6443336_6444923,AGCGATAGAATACGATAATGGCAGCTCGCAGGACGT,2
chr1_6584548_6585071,AGCGATAGAATACGATAATGGCAGCTCGCAGGACGT,1
chr1_7127882_7129914,AGCGATAGAATACGATAATGGCAGCTCGCAGGACGT,2
chr1_7136712_7137602,AGCGATAGAATACGATAATGGCAGCTCGCAGGACGT,1


In [7]:
input_cds <- make_atac_cds(cicero_data, binarize = TRUE)

In [8]:
dim(input_cds)

Features  Samples 
  385237    12178

In [9]:
#Ensure there are no peaks included with zero reads
input_cds <- input_cds[Matrix::rowSums(exprs(input_cds)) != 0,] 

In [10]:
dim(input_cds)

Features  Samples 
  385237    12178

In [11]:
input_cds

CellDataSet (storageMode: environment)
assayData: 385237 features, 12178 samples 
  element names: exprs 
protocolData: none
phenoData
  sampleNames: AGCGATAGAACGAATTCGAAGCCTACGACCTATCCT
    AGCGATAGAACGAATTCGAAGCCTACGATATAGCCT ...
    TCTCGCGCTTGGTAAGGATTGGTAGTCGTATAGCCT (12178 total)
  varLabels: cells Size_Factor num_genes_expressed
  varMetadata: labelDescription
featureData
  featureNames: chr1_3002715_3002962 chr1_3037090_3037634 ...
    chrY_631121_631487 (385237 total)
  fvarLabels: site_name chr ... num_cells_expressed (5 total)
  fvarMetadata: labelDescription
experimentData: use 'experimentData(object)'
Annotation:  

In [12]:
set.seed(2019)

In [13]:
input_cds <- detectGenes(input_cds)

In [14]:
input_cds <- estimateSizeFactors(input_cds)

In [15]:
input_cds <- reduceDimension(input_cds, max_components = 2, num_dim=15,
                        reduction_method = 'tSNE', norm_method = "none")

Warning message in int2i(as.integer(i), n):
"NAs introduced by coercion to integer range"

ERROR: Error in if (any(i < 0L)) {: missing value where TRUE/FALSE needed


In [16]:
sessionInfo()

R version 3.5.1 (2018-07-02)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_Cicero/lib/R/lib/libRblas.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
 [1] grid      splines   stats4    parallel  stats     graphics  grDevices
 [8] utils     datasets  methods   base     

other attached packages:
 [1] umap_0.2.2.0         reshape2_1.4.3       proxy_0.4-23        
 [4] data.table_1.12.0    cicero_1.0.15        Gviz_1.26.5         
 [7] GenomicRanges_1.34.0 GenomeInfoDb_1.18.1  IRanges_2.16.0      
[10] S4Vectors_0.20.1     mo